In [1]:
import Naked as nk
from Naked.toolshed.shell import execute_js, muterun_js,execute
import os
import json

In [282]:
import ast
#Requires snyk1.js to be placed in the same folder as the notebook

def prase_snyk(arguments='numpy'):
    
    struct = muterun_js(file_path='./snyk1.js',arguments=arguments)
    
    output = struct.stdout
    errors = struct.stderr
    
    #handling errors...
    if errors != b'':
        print('There is an error in a JS part: '+errors)
        return 
    if b'UnhandledPromiseRejectionWarning' in output:
        print('Unhandled Promise Rejection Warning - module information is unavailable')
        return []
    if struct.exitcode != 0:
        print('Exitcode is nonzero - check the input!')
        return
    
    #creating a list of vulnerabilities, where each element is a dict, containing vulnerability features
    feats = ['ok:','title:','moduleName:','packageName:','language:','packageManager:','description:','severity:','cvssScore:','CVSSv3:','creationTime:','modificationTime:','publicationTime:','disclosureTime:','id:','version:','name:','isUpgradable:','isPatchable:']
    summary_feats = ['uniqueCount:','summary:','severityMap:','numDependencies:']

    result = []
    splitted = (struct.stdout).decode('utf8').splitlines(0)
    while " ".join(splitted).count('cvssScore:')>0:
        d = {}
        for feat in feats:
            for line in splitted:
                raw_line = line
                line = line.replace('}','').replace('{','')
                if feat in line:
                    d[feat[:-1]] = line[line.find(':')+2:-1]
                    if feat[:-1] not in 'ok, isPatchable, isUpgradable': # boolean valule outputs
                        d[feat[:-1]] = ast.literal_eval(d[feat[:-1]])
                    else:
                        d[feat[:-1]] = d[feat[:-1]].replace(' ','')
                    splitted.remove(raw_line)
                    break
        result.append(d)
        
    #overwhole statistics for the module
    summary = {}
    for feat in summary_feats:
        for line in splitted:
            if feat in line:
                summary[feat[:-1]] = line[line.find(':')+2:-1]
                if feat[:-1] == 'summary':
                    summary[feat[:-1]] = summary[feat[:-1]][1:-1]
    return result, summary

In [283]:
res,summ = prase_snyk('scipy')